## Notebook JetCar_Collect_Images
### Collect Scene Images for ImageSegmenter 
> This notebook allows manually driving the car around to take images of the scene
***

<br>
Import major libraries for this notebook

In [ ]:
import os
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from utils import preprocess,load_color_map
import numpy as np
from jetcar_definitions import *

Import JetCar hardware class and instantiate car object

In [2]:
from jetcar_car import JetCar
car = JetCar()

Import and instantiate camera object with full field of view scaled down to 4 times image size for ImageSegementer

In [3]:
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=4*IMG_SIZE, height=4*IMG_SIZE, capture_width=3280, capture_height=2464, capture_fps=4)

Create widgets for throttle and steering control

In [4]:
import ipywidgets.widgets as widgets

max_throttle = 0.3
throttle_enable_widget = ipywidgets.Checkbox(value=True, description='Throttle Enable',layout={'width': '200px'} )
steering_value_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.001, value=0, description='Steering', layout={'width': '900px'})
throttle_value_slider = ipywidgets.FloatSlider(min=-max_throttle, max=max_throttle, step=0.001, value=0, description='Throttle',layout={'width': '900px'})

def on_throttle_enable_change(change):
    if change['new'] == True:
        car.throttle = throttle_value_slider.value
    else:
        car.throttle = 0
throttle_enable_widget.observe(on_throttle_enable_change, names='value')

def on_steering_value_slider_change(change):
    car.steering = change['new']
steering_value_slider.observe(on_steering_value_slider_change, names='value')

def on_throttle_value_slider_change(change):
    if throttle_enable_widget.value == True:
        car.throttle = change['new']
    else:
        car.throttle = 0
throttle_value_slider.observe(on_throttle_value_slider_change, names='value')

Create widgets for capture control and ending the collection

In [5]:
status_text_widget = ipywidgets.Text(value='', layout={'width': '700px'})
capture_button = ipywidgets.Button(description="Capture", layout={'width': '700px'})
stop_button = ipywidgets.Button(description="Stop", layout={'width': '200px'})

Create camera and snapshot image widgets

In [6]:
WIDGET_SIZE=2*IMG_SIZE
camera_widget = ipywidgets.Image(width=WIDGET_SIZE, height=WIDGET_SIZE)
snapshot_widget = ipywidgets.Image(width=WIDGET_SIZE, height=WIDGET_SIZE)

Define the layout of all widgets on the screen

In [7]:
display_widgets = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]), 
    steering_value_slider,
    throttle_value_slider,
    ipywidgets.HBox([status_text_widget,throttle_enable_widget]), 
    ipywidgets.HBox([capture_button, stop_button])
])

Make sure the target folder exists but don't delete existing images. 
Define a function to create an identifier and the file name for the next image

In [8]:
image_dir = "Images"
    
if os.path.exists(image_dir) == False:
    os.mkdir(image_dir)

def update_image_id():
    global img_id,file_name,image_path
    img_id = len([entry for entry in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, entry))])
    while True:
        file_name = "Image_%03d.jpg" % (img_id)
        if os.path.exists(file_name) == False:
            break
        img_id += 1

    image_path = os.path.join(image_dir, file_name)
    status_text_widget.value = "Next: %s" % (image_path)
    return

update_image_id()

Define the event handler for the cpature button click to store ethe image

In [9]:
def on_capture_button_click(_):
    snapshot = camera.value.copy()
    snapshot_widget.value = bgr8_to_jpeg(snapshot)
    cv2.imwrite(image_path, snapshot)
    update_image_id()
    return

capture_button.on_click(on_capture_button_click)

Define the event handler for the stop button to stop the car, shut down camera and zip up all images

In [10]:
def on_stop_button_click(_):
    global car,camera
    
    camera.running = False
    car.throttle = 0
    car.steering = 0
    
    try:
        camera.unobserve_all()
    except:
        print("Exception at camera.unobserve_all()")
        
    try:
        del camera
    except:
        print("Exception at del camera")

    try:
        if os.path.exists("Images.zip") == True:
            os.remove("Images.zip")
        !zip -r Images.zip {image_dir}
    except:
        print("Exception at zipping images") 
    return

stop_button.on_click(on_stop_button_click)

Turn on the camera and link the real-time updates to the camera widget.
Then display all widgets on the screen.
Move the car around manually and capture images for the ImageSegmenter.

In [11]:
camera.running = True
camera.unobserve_all()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

display(display_widgets)

When done, hit the stop button. All images in the folder will be zipped up into Images.zip, ready for download.